<div align="center">
    <h1>DS-210: Programming for Data Science</h1>
    <h1>Lecture 17</h1>
</div>

1. Iterators
2. Iterator + closure magic
3. How about Python?
4. Error handling in Rust
5. Binary search 

## Iterators

> The iterator pattern allows you to perform some task on a sequence of items in turn. 
> 
> An iterator is responsible for the logic of iterating over each item and determining when the sequence has finished.

* provide values one by one
* method `next` provides next one
* `Some(value)` or `None` if no more available

Some ranges are iterators:
* `1..100`
* `0..`

First value has to be known (so `..` and `..123` are not)

In [2]:
let mut iter = 1..3; // must be mutable

In [3]:
println!("{:?}", iter.next());
println!("{:?}", iter.next());
println!("{:?}", iter.next());
println!("{:?}", iter.next());
println!("{:?}", iter.next());

Some(1)
Some(2)
None
None
None


* What about a range between floats?

In [4]:
let mut iter = 1.0..3.0; // must be mutable
println!("{:?}", iter.next());
println!("{:?}", iter.next());
println!("{:?}", iter.next());
println!("{:?}", iter.next());
println!("{:?}", iter.next());

Error: the method `next` exists for struct `Range<{float}>`, but its trait bounds were not satisfied

Error: the method `next` exists for struct `Range<{float}>`, but its trait bounds were not satisfied

Error: the method `next` exists for struct `Range<{float}>`, but its trait bounds were not satisfied

Error: the method `next` exists for struct `Range<{float}>`, but its trait bounds were not satisfied

Error: the method `next` exists for struct `Range<{float}>`, but its trait bounds were not satisfied

* In Rust, ranges over floating-point numbers (f64) are not directly iterable. 

* This is because floating-point numbers have inherent precision issues that make it difficult to guarantee exact iteration steps.

* But this works.

In [5]:
let mut iter = 'a'..'c'; // must be mutable
println!("{:?}", iter.next());
println!("{:?}", iter.next());
println!("{:?}", iter.next());
println!("{:?}", iter.next());
println!("{:?}", iter.next());

Some('a')
Some('b')
None
None
None


## Iterator from scratch: implement trait `Iterator`

In [6]:
struct Fib {
    current: u128,
    next: u128,
}

impl Fib {
    fn new() -> Fib {
        Fib{current: 0, next: 1}
    }
}

impl Iterator for Fib {
    type Item = u128;
    
    fn next(&mut self) -> Option<Self::Item> {
        let now = self.current;
        self.current = self.next;
        self.next = now + self.current;
        Some(now)
    }
}

In [9]:
let mut fib = Fib::new();
for _ in 0..10 {
    print!("{:?} ",fib.next().unwrap());
}
println!();

0 1 1 2 3 5 8 13 21 34 


## Iterators come with many useful methods implemented

Pay special attention to what the output is.

* `next()` -> Get the next element of an iterator (None if there isn't one)
* `collect()` -> Put iterator elements in collection
* `take(N)` -> take first N elements of an iterator and _turn them into an iterator_
* `cycle()` -> Turn a finite iterator into an infinite one that repeats itself
* `for_each(||, )` -> Apply a closure to each element in the iterator
* `filter(||, )` -> _Create new iterator_ from old one for elements where closure is true
* `map(||, )` -> _Create new iterator_ by applying closure to input iterator
* `any(||, )` -> Return true if closure is true for any element of the iterator
* `fold(a, |a, |, )` -> Initialize expression to a, execute closure on iterator and accumulate into a
* `reduce(|x, y|, )` -> Similar to fold but the initial value is the first element in the iterator
* `zip(iterator)` -> Zip two iterators together to turn them into pairs

If the method returns an iterator, you have to do something with the iterator.

In [27]:
// this does nothing!
{
    let v1 = vec![1, 2, 3];

    let v1_iter = v1.iter();
}


()

`collect` can be used to put elements of an iterator into a vector:

In [10]:
let small_numbers : Vec<_> = (1..=10).collect();
small_numbers

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

`take` turns an iterator into an iterator that provides at most a specific number of elements

In [3]:
let small_numbers : Vec<_> = (1..).take(30).collect();
small_numbers

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]

`cycle` creates an iterator that repeats itself forever:

In [13]:
let cycle : Vec<_> = (1..4).cycle().take(20).collect();
cycle

[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2]

## Iterator + closure magic
* Operate on entire sequence, sometimes lazily by creating a new iterator
* Allows for concise expression of many concepts

`for_each` applies a function to each element

In [60]:
(0..5).for_each(|x| println!("{}",x));

0
1
2
3
4


`filter` creates a new iterator that has elements for which the given function is true

In [61]:
let not_divisible_by_3 : Vec<_> = (0..10).filter(|x| x % 3 != 0).collect();
not_divisible_by_3

[1, 2, 4, 5, 7, 8]

## Iterator + closure magic
* Operate on entire sequence, sometimes lazily by creating a new iterator
* Allows for concise expression of many concepts


`map` creates a new iterator in which values are processed by a function

In [14]:
let fibonacci_squared : Vec<_> = Fib::new().take(10).map(|x| x*x).collect();
fibonacci_squared

[0, 1, 1, 4, 9, 25, 64, 169, 441, 1156]

## Primes
`any` is true if the passed function is true on some element

Is a number prime?

In [4]:
fn is_prime(k:u32) -> bool {
    !(2..k).any(|x| k % x == 0)
}

In [7]:
is_prime(33)

false

Create infinite iterator over primes:

In [10]:
{
  let primes = (2..).filter(|k| !(2..*k).any(|x| k % x == 0));  // create a new iterator
  let v : Vec<_> = primes.take(20).collect();
  v
}

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71]


## Functional programming classics: `fold` and `reduce`

* `fold(a, |a, |, )` -> Initialize expression to a, execute closure on iterator and accumulate into a

<br>

`iterator.fold(init,f)` equivalent to

```rust
let mut accumulator = init;
while let Some(x) = iterator.next() {
    accumulator = f(accumulator,x);
}
accumulator
```

**Example:** compute $\sum_{i=1}^{10} x^2$

In [11]:
let sum_of_squares: i32 = (1..=4).fold(0,|a,x| a + x * x);
sum_of_squares

30

In [12]:
// Another approach: using `sum` (which can be implemented using `fold`)
let sum_of_squares: i32 = (1..=10).map(|x| x * x).sum();
sum_of_squares

385

## Functional programming classics: `fold` and `reduce`

* `reduce(|x, y|, )` -> Similar to fold but the initial value is the first element in the iterator

<br>

`iterator.reduce(f)` equivalent to
```rust
if let Some(x) = iterator.next() {
    let mut accumulator = x;
    while let Some(y) = iterator.next() { accumulator = f(accumulator,y}
    Some(accumulator)
} else {
    None
}
```

Differences from `fold`:
* no default value for an empty sequence 
* output must be the same type as elements of input sequence
* output for length–one sequence equals the only element in the sequence

**Example:** computing the maximum number in $\{x^2 \bmod 7853: x \in [1,...,123] \}$

In [24]:
(1..=123).map(|x| (x*x) % 7853).reduce(|x,y| x.max(y)).unwrap()

7744

In [25]:
// in this case one can use the builtin `max` method (which can be implemented, using `fold`)
(1..=123).map(|x| (x*x) % 7853).max().unwrap()

7744

## Combining two iterators: `zip`

* Returns an iterator of pairs
* The length is the minimum of the lengths

In [26]:
let v: Vec<_>= (1..10).zip(11..20).collect();
v

[(1, 11), (2, 12), (3, 13), (4, 14), (5, 15), (6, 16), (7, 17), (8, 18), (9, 19)]

Inner product of two vectors:

In [27]:
let x: Vec<f64> = vec![1.1,  2.2, -1.3,  2.2];
let y: Vec<f64>  = vec![2.7, -1.2, -1.1, -3.4];
let inner_product: f64 = x.iter().zip(y.iter()).map(|(a,b)| a * b).sum();
inner_product

-5.72


# <font color="red">How about Python?</font>

<div align="center">
    <h1>[Switch to the Python notebook]</h1>
</div>

## Error handling in Rust

Two basic options:

* terminate when an error occurs: macro `panic!(...)`

* pass information about an error: enum `Result<T,E>`

## Macro `panic!(...)`

* Use for unrecoverable errors
* Terminates the application

In [28]:
fn divide(a:u32, b:u32) -> u32 {
    if b == 0 {
        panic!("I'm sorry, Dave. I'm afraid I can't do that.");
    }
    a/b
}

In [29]:
divide(20,7)

2

In [30]:
divide(20,0)

thread '<unnamed>' panicked at src/lib.rs:28:9:
I'm sorry, Dave. I'm afraid I can't do that.
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: <unknown>
   3: evcxr::runtime::Runtime::run_loop
   4: evcxr::runtime::runtime_hook
   5: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


## Enum `Result<T,E>`

```rust
enum Result<T,E> {
    Ok(T),
    Err(E),
}
```

Functions can use it to
* return a result
* or information about an encountered error

In [13]:
fn divide(a:u32, b:u32) -> Result<u32, String> {
    if b != 0 {
        Ok(a / b)
    } else {
        let str = format!("Division by zero {} {}", a, b);
        Err(str)
    }
}

In [14]:
divide(20,7)

Ok(2)

In [15]:
divide(20,0)

Err("Division by zero 20 0")

* Useful when the error best handled somewhere else
* **Example:** input/output subroutines in the standard library

## Common pattern: propagating errors

* We are interested in the positive outcome: `t` in `Ok(t)`
* But if an error occurs, we want to propagate it
* This can be handled using `match` statements

In [16]:
// compute a/b + c/d
fn calculate(a:u32, b:u32, c:u32, d:u32) -> Result<u32, String> {
    let first = match divide(a,b) {
        Ok(t) => t,
        Err(e) => return Err(e),
    };
    let second = match divide(c,d) {
        Ok(t) => t,
        Err(e) => return Err(e),
    };    
    Ok(first + second)
}


In [17]:
calculate(16,4,18,3)

Ok(10)

In [18]:
calculate(16,0,18,3)

Err("Division by zero 16 0")

## The question mark shortcut

* Place `?` after an expression that returns `Result<T,E>`

* This will:
  * give the content of `Ok(t)`
  * or return `Err(e)` from the encompassing function

In [19]:
// compute a/b + c/d
fn calculate(a:u32, b:u32, c:u32, d:u32) -> Result<u32, String> {
    Ok(divide(a,b)? + divide(c,d)?)
}

In [20]:
calculate(16,4,18,3)

Ok(10)

In [21]:
calculate(16,0,18,3)

Err("Division by zero 16 0")

### Error handling summary

* In some languages we have the pattern try/catch or throw/catch or try/except (C++, Java, Javascript, Python).
* Rust does not have something equivalent

The Rust pattern for error handling is the following:
```
    let do_steps = || -> Result<(), MyError> {
        do_step_1()?;
        do_step_2()?;
        do_step_3()?;
        Ok(())
    };

    if let Err(_err) = do_steps() {
        println!("Failed to perform necessary steps");
    }
```

* Create a closure with the code you want to guard.  Use the ? shorthand inside the closure for anything that can return an Error.  Use a match or if let statement to catch the error.

## Binary search

**Input:** sorted array/vector `v`

**Task:** find if a specific element `x` appears in it

General strategy:
* maintain range `left..right` within `v` of where `x` could be
* repeat:
   * ask about the middle of the range
   * if `x` found, celebrate
   * if the middle element ${}<{}$ `x`, narrow search to the right half
   * otherwise, narrow search to the left half

## Let's implement it!

**Might be harder than you think!**

Jon Bentley in "Programming pearls" claims that only 10% programmers succeed, even given unlimited amount of time

**Ad:**
 * Great book if you are interested in programming and basic algorithms
 * First edition mostly uses pseudocode
 * 2nd edition focused on C++ (probably still great)
 * very cheap if you want to buy second hand (< \$10)
 

## Let's implement it!

In [40]:
fn present(mut data:&[i32], element:i32) -> Option<(usize,i32)> {
    let (mut left, mut right) = (0, data.len());
    // invariant: if element has not been found, it must be in data[left..right]
    while left < right {
        let middle = (left+right)/2;
        if data[middle] == element {
            return Some((middle, element));
        } else if data[middle] > element {
            right = middle - 1 
        } else {
            left = middle + 1
        }
    }
    None
}

In [41]:
let v = vec![-16,-4,0,2,4,12,32,48,48,111];

In [42]:
present(&v, 111)

Some((9, 111))

In [43]:
present(&v, 5)

None

In [44]:
present(&v, 1000)

None

In [45]:
present(&v, 32)

Some((6, 32))